#Install requirements

In [1]:
!pip install -q langgraph langchain langchain_openai langchainhub langchain-core langchain_experimental

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataprep 0.4.5 requires pandas<2.0,>=1.1, but you have pandas 2.2.1 which is incompatible.
dataprep 0.4.5 requires pydantic<2.0,>=1.6, but you have pydantic 2.6.4 which is incompatible.
dataprep 0.4.5 requires regex<2022.0.0,>=2021.8.3, but you have regex 2023.12.25 which is incompatible.
dataprep 0.4.5 requires sqlalchemy==1.3.24, but you have sqlalchemy 2.0.27 which is incompatible.
modin 0.20.1 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.
streamlit 1.20.0 requires pandas<2,>=0.25, but you have pandas 2.2.1 which is incompatible.
streamlit 1.20.0 requires protobuf<4,>=3.12, but you have protobuf 4.25.3 which is incompatible.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.4 which is incompatible.
transformers 4.24.0 requires tokenizers!=0.11.3,<0.14,>=0.11.

#Load key

In [2]:
import os
import getpass

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

#Define the Tools
### tools = function call

In [4]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool
from langgraph.prebuilt import ToolExecutor
from langchain_core.tools import tool

In [5]:
@tool
def multiply(first_number: int, second_number: int) -> int:
    """Multiplies two numbers together."""
    return first_number * second_number

In [6]:
tools = [multiply]

# Set up the agent

In [7]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [8]:
model = ChatOpenAI(model="gpt-3.5-turbo",
                   temperature=0,
                   streaming=True)
system_template = """
You are a assistant.
"""
human_template = "{input}"

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_template),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate.from_template(
            input_variables=["input"], template=human_template
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [10]:
agent = create_openai_tools_agent(model, tools, prompt)

#Run it

In [11]:
from langchain.agents import AgentExecutor

In [12]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_config({"run_name": "Agent with Tools"})

In [18]:
agent_executor.invoke(
    {
        "input": "what is 2 mul 8?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'first_number': 2, 'second_number': 8}`


16The result of multiplying 2 by 8 is 16.

> Finished chain.


{'input': 'what is 2 mul 8?',
 'output': 'The result of multiplying 2 by 8 is 16.'}